# ML-Brawlhalla: Trial 2

## Import Packages
The environment module is not included in this code, so that results cannot be abused.

In [ ]:
from tensorflow import keras

from paiutils import neural_network as nn
from paiutils import reinforcement as rl

from environment import BrawlhallaFollow as Brawlhalla

## Create Environment

In [ ]:
env = Brawlhalla(stack_size=3)
max_steps = 100000

## Create the Model

In [ ]:
inputs = keras.layers.Input((64, 64, 3))
x = nn.conv2d(32, 5, 2)(inputs)
x = nn.conv2d(64, 3, 2)(x)
x = nn.conv2d(128, 3, 2)(x)
x = nn.conv2d(256, 3, 2)(x)
x = nn.conv2d(512, 3, 2)(x)
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(7, activation='softmax')(x)
model = keras.models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(.0002), loss='mse')
model.summary()

## Create the Agent

In [ ]:
def create_memory(shape, dtype):
    return rl.Memory()

agent = rl.PGAgent(model, .99, create_memory=create_memory)

## Train the Agent

In [ ]:
agent.set_playing_data(memorizing=True, verbose=True)
env.play_episodes(agent, 5, max_steps, random=True,
                  verbose=True, episode_verbose=True)

agent.set_playing_data(
    training=True, memorizing=True, batch_size=64,
    mini_batch=12800, epochs=1, repeat=1,
    entropy_coef=0, verbose=True
)
save_dir = 'saves/'
num_episodes = 10
for ndx in range(8):
    print(f'Save Loop: {ndx}')
    result = env.play_episodes(
        agent, num_episodes, max_steps,
        verbose=True, episode_verbose=True
    )
    agent.save(save_dir, note=f'PG_{ndx}_{result}')

## Test the Agent

In [ ]:
agent.load(save_dir + 'pg_80', load_data=False)
agent.set_playing_data(training=False, memorizing=False)
env.play_episodes(agent, 20, 10000, verbose=True,
                  episode_verbose=False)